In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    'figure.figsize': (7, 5),
    'figure.dpi': 100,
    'text.usetex': True,
    'font.family': 'serif',
    'font.size': 12,
})
# Read in the data
df = pd.read_csv('gil_20_tasks.csv', sep=', ', engine='python')

df_frac0 = df[df['frac'] == 0.1]
print(df_frac0)


      workers   n  task_time  accesses  frac  total_time
3           1  20        100         1   0.1    0.003078
12          2  20        100         1   0.1    0.001879
21          3  20        100         1   0.1    0.001520
30          4  20        100         1   0.1    0.001483
39          5  20        100         1   0.1    0.001871
...       ...  ..        ...       ...   ...         ...
3846        4  20      13000       100   0.1    0.069266
3855        5  20      13000       100   0.1    0.056169
3864        6  20      13000       100   0.1    0.057883
3873        7  20      13000       100   0.1    0.051577
3882        8  20      13000       100   0.1    0.059606

[432 rows x 6 columns]


In [2]:
def plot_speedup(df, accesses, frac):

    fig, ax = plt.subplots()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    #Compute expected time
    df['expected'] = (df['task_time']*df['n'])/(1e6)

    #Select the data for the given accesses and frac
    df = df[(df['accesses'] == accesses) & (df['frac'] == frac)]
    print(df.head())
    plt.plot(df['workers'], df['expected']/df['total_time'], label='Speedup')

    plt.xlabel('Workers')
    plt.ylabel('Speedup')


In [3]:
def plot_fraction_workers(df, time, accesses):

    from scipy.interpolate import griddata

    #Plot as 2D surface

    fig, ax = plt.subplots()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    #Compute expected time
    df['expected'] = (df['task_time']*df['n'])/(1e6)
    df['speedup'] = df['expected']/df['total_time']

    #Select the data for the given accesses and frac
    df = df[(df['task_time'] == time) & (df['accesses'] == accesses)]

    #print(df[df['frac'] == 0])

    workers = df['workers'].unique()
    fracs = df['frac'].unique()
    workers, fracs = np.meshgrid(workers, fracs)
    workers = workers.T
    fracs = fracs.T
    speedup  = df['speedup'].values.reshape(workers.shape)
    
    #Interpolate the data
    #zi = griddata((df['workers'], df['accesses']), df['expected']/df['total_time'], (np.arange(1, 9), np.arange(1, 11)), method='linear')
    mesh = plt.contourf(workers, fracs, speedup, levels=np.arange(0, 9), cmap='RdBu', vmin=1, vmax=8)
    ax.set_xlim([1, 8])
    ax.set_ylim([0, 0.5])

    #print(df['workers'])
    ax.set_xlabel('Workers')
    ax.set_ylabel('GIL Held \%')
    plt.colorbar(mesh)


#plot_fraction_workers(df, 50000, 1)

In [4]:
def plot_access_workers(df, time, fraction):

    from scipy.interpolate import griddata

    #Plot as 2D surface

    fig, ax = plt.subplots()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    #Compute expected time
    df['expected'] = (df['task_time']*df['n'])/(1e6)
    df['speedup'] = df['expected']/df['total_time']

    #Select the data for the given accesses and frac
    df = df[(df['task_time'] == time) & (df['frac'] == fraction)]

    print(df.head())
    #print(df[df['frac'] == 0])

    workers = df['workers'].unique()
    fracs = df['accesses'].unique()
    workers, fracs = np.meshgrid(workers, fracs)
    workers = workers
    fracs = fracs
    speedup = df['speedup'].values.reshape(workers.shape)

    #Interpolate the data
    #zi = griddata((df['workers'], df['accesses']), df['expected']/df['total_time'], (np.arange(1, 9), np.arange(1, 11)), method='linear')
    mesh = plt.contourf(workers, fracs, speedup, levels=np.arange(
        0, 9), cmap='RdBu', vmin=1, vmax=8)
    ax.set_xlim([1, 8])
    ax.set_ylim([0, 100])

    #print(df['workers'])
    ax.set_xlabel('Workers')
    ax.set_ylabel('GIL Accesses')
    plt.colorbar(mesh, label='Speedup')


In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

drop_time = widgets.Dropdown(options=df['task_time'].unique(), value=df['task_time'].values[0], description='Task Length:')
drop_accesses = widgets.Dropdown(options=df['accesses'].unique(), value=1, description='Accesses:')

interact(plot_fraction_workers, df=fixed(df), time=drop_time, accesses=drop_accesses)

interactive(children=(Dropdown(description='Task Length:', options=(100, 500, 1000, 3000, 5000, 7000, 9000, 11…

<function __main__.plot_fraction_workers(df, time, accesses)>

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

drop_time = widgets.Dropdown(options=df['task_time'].unique(
), value=df['task_time'].values[0], description='Task Length:')
drop_frac = widgets.Dropdown(
    options=df['frac'].unique(), value=0, description='GIL Fraction:')

interact(plot_access_workers, df=fixed(df),
         time=drop_time, fraction=drop_frac)


interactive(children=(Dropdown(description='Task Length:', options=(100, 500, 1000, 3000, 5000, 7000, 9000, 11…

<function __main__.plot_access_workers(df, time, fraction)>